# Mastering Applied Skills in Management, Analytics and Entrepreneurship I

## DATA COLLECTION TECHNIQUES
## Part I. Connect to remote database

### 1. Connect to PostgreSQL database

In [ ]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor

[Psycopg](https://www.psycopg.org/docs/index.html) is already installed in the JupyterHub environment as a PostgreSQL database adapter for Python.

In [ ]:
# do not do like this!
POSTGRESQL_HOST = 'myhost'
DBNAME = 'mydbase'
USERNAME = 'mynamr'
PASSWORD = 'youshallnotpass'

In [ ]:
os.listdir()

In [ ]:
# this is better
def access_data(file_path):
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data

creds = access_data(file_path='access_dbase.json')
print(creds.keys())

__NOTE:__ the best solution is to use environment variables. About [environment variables](https://help.ubuntu.com/community/EnvironmentVariables) and [how to use it](https://www.twilio.com/blog/environment-variables-python).

In [ ]:
conn = psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
)
cur = conn.cursor()

### 2. How to send query

In [ ]:
query = 'SELECT * FROM pg_catalog.pg_tables;'

In [ ]:
cur.execute(query)
records = cur.fetchall()
cur.close()
conn.close()

In [ ]:
print(records[0])

### 3. How to send query ('with' context manager)

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        records = cur.fetchall()

In [ ]:
print(records[0])

In [ ]:
conn, cur

### 4. Multiply queries

In [ ]:
queries = {
    '___DATABASES___': 'SELECT * FROM pg_database',
    '___TABLES___': "SELECT relname FROM pg_class WHERE relkind='r' AND relname !~ '^(pg_|sql_)';",
    '\n___COLUMNS___': "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'seats';"
}

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    for name, query in queries.items():
        print('\n', name)
        with conn.cursor() as cur:
            cur.execute(query)
            for row in cur.fetchall():
                print(row)

### 5. SQL for IPython

The [IPython SQL](https://github.com/catherinedevlin/ipython-sql) framework allows to use [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) like `%sql` (or `%%sql`) for SQL queries. We can connect to a database, then issue SQL commands within Jupyter.

In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql

Connection data string to be used later:

In [ ]:
CONNECT_DATA = 'postgresql://{}:{}@{}:{}/{}'.format(
    creds['USERNAME'],
    creds['PASSWORD'], 
    creds['POSTGRESQL_HOST'],
    '6432',
    'dbsimba'
)
CONNECT_DATA

Let's look at all databases in PostgreSQL. SQL query can be done after the connection with `%%sql` magic command:

In [ ]:
%%sql $CONNECT_DATA
    SELECT * FROM public.jhublogs LIMIT 10

In [ ]:
%sql SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

## <font color='red'>INTERMEDIATE QUIZ</font>
Answer the following questions with the help of `%%sql`:
1. How many logs are there? (HINT: use `COUNT` command)
2. What time interval is covered by logs? (HINT: use `max` and `min` commands over `log_timestamp` column)
3. Find all types of logs (HINT: `log_type` columns)
4. Find all logs that are produced by your activity? (HINT: use `LIKE '%mylogin%'` syntax)

### 6. How to deal with the data

In [ ]:
query = 'SELECT * FROM public.jhublogs LIMIT 100000;'

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        # names of table columns
        col_names = [desc[0] for desc in cur.description]
        # data from table
        records = cur.fetchall()

In [ ]:
print(records[:3])

In [ ]:
print(colnames)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(records, index=None)
df.columns = col_names
df.head()

### 7. ClickHouse connection demo

We will need very famous [Requests](https://requests.readthedocs.io/en/latest/index.html) library.

In [ ]:
import os
import requests
import pandas as pd

In [ ]:
CH_HOST = 'http://10.129.0.30'
CH_PORT = '8123'
SSL_VERIFY = True
query = 'SHOW DATABASES'
user_name=os.environ['CLICKHOUSE_USER']
user_passwd=os.environ['CLICKHOUSE_PASSWORD']

In [ ]:
r = requests.post(
    url=':'.join([CH_HOST, CH_PORT]), 
    data=query,
    auth=(
        os.environ['CLICKHOUSE_USER'], 
        os.environ['CLICKHOUSE_PASSWORD']
    ), 
    verify=SSL_VERIFY
)
print('request status code:', r.status_code)

In [ ]:
r.text

In [ ]:
def get_data(query, host, user_name, user_passwd):
    if (user_name == '') and (user_passwd == ''):
        r = requests.post(
            host, 
            params=query_dict, 
            verify=SSL_VERIFY
        )
    else:
        r = requests.post(
            host, 
            data=query,
            auth=(
                user_name, 
                user_passwd
            ), 
            verify=SSL_VERIFY
        )
    print('request status code:', r.status_code)
    return r.text

In [ ]:
query = 'SHOW TABLES FROM gsomlogs'
data = get_data(
    query=query, 
    host=':'.join([CH_HOST, CH_PORT]),
    user_name=os.environ['CLICKHOUSE_USER'],
    user_passwd=os.environ['CLICKHOUSE_PASSWORD']
)

In [ ]:
data

In [ ]:
data = [x.split('\t') for x in data.split('\n')]
pd.DataFrame(data)

In [ ]:
query = 'SELECT * FROM gsomlogs.hits_all ORDER BY DateTime DESC LIMIT 100'
data = get_data(
    query=query, 
    host=':'.join([CH_HOST, CH_PORT]),
    user_name=os.environ['CLICKHOUSE_USER'],
    user_passwd=os.environ['CLICKHOUSE_PASSWORD']
)

In [ ]:
data = [x.split('\t') for x in data.split('\n')]
df = pd.DataFrame(data)
df.head()

## LAB WORK #1

Your lab work is as follows:
1. Take Jupyter logs from database (PostgreSQL)
2. Explore them 
3. Find users that log in Jupyter most often

__HINT:__ check fields `log_code` or `log_type` to undercover log in activities